### ¿Qué genera este script?

Para cada sitio y fecha, el total de usuarios registrados (usuarios.usuariosRegistrados)

In [1]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re
import random
import time

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [3]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy e indice de control

### Función para generar JSON compatible con ElactiSearch

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISOid_Beneficiario).

In [6]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [7]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'

In [8]:
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [9]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ['usuarios.fechaControl'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero12"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-07 00:00:00


### Asociando datos de usuarios

* Se realiza lectura por día procesado
* lugar_cod se usa para cruzar con semilla por site_id
* creado es la fecha de creación del usuario
* tipodoc y documento corresponden a datos del usuario. Se usa para calcular totales

In [10]:
def traeRegistros(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.ohmyfi_d_u_index,
        body={
                "_source": ['lugar_cod','tipodoc', 'documento', 'creado']
                  , "query": {
                  "range": {
                    "creado": {
                      "gte": fecha_max.split(' ')[0]+' 00:00:00',
                      "lt": fecha_tope.split(' ')[0]+' 23:59:59'
                      #"gte": "2021-05-26 10:00:00",
                      #"lt": "2021-05-26 15:00:00"  
                    }
                  }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#    fields = {}
#    for num, doc in enumerate(elastic_docs):
#        source_data = doc["_source"]
#        for key, val in source_data.items():
#            try:
#                fields[key] = np.append(fields[key], val)
#            except KeyError:
#                fields[key] = np.array([val])

#    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Se realiza consulta de datos

* Se calcula rango en base a la fecha de control. Para este caso es de un día.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha actual.

In [11]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_registro = traeRegistros(fecha_max_mintic,fecha_tope_mintic)

if datos_registro is None or datos_registro.empty:
    while (datos_registro is None or datos_registro.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_speed = traeRegistros(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

# leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR

site_id para cruzar con las misma llave de semilla.
datos del dispositivo: mac, status, ip.
ap_group para identificar si la conexión es indoor/outdoor

In [12]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","status","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios ya que no cruzarán en el merge y se limpian los NaN del dataframe

In [13]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

Se corrigen datos de ap group con formato no valido

In [14]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split(".", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos con site_id y mac unicos

In [15]:
#datos_dev.drop_duplicates(subset=['site_id','mac'],inplace=True)
datos_dev = datos_dev.drop_duplicates('mac')

Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario

In [16]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

# Se lee información de Ohmyfi consumos

Apartir de esta lectura se toma el valor del tiempo promedio de sesión en minutos. Se toma de referencia los campos:

fecha_inicio a partir de la cual se calcula fecha control
tiempo_sesion_minutos
mac_usuario
lugar_cod

In [17]:
def traeSesiones(fecha_max,fecha_tope):
    total_docs = 500000
    response = es.search(
        index= parametros.ohmyfi_consumos_index,
        body={
                  "_source": ["lugar_cod", "tiempo_sesion_minutos","mac_usuario","fecha_inicio"]
                , "query": {
                      "range": {
                            "fecha_inicio": {
                            "gte": fecha_max,
                            "lt": fecha_tope
                            }
                        }
                  }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    #fields = {}
    #for num, doc in enumerate(elastic_docs):
        #source_data = doc["_source"]
        #for key, val in source_data.items():
            #try:
                #fields[key] = np.append(fields[key], val)
            #except KeyError:
                #fields[key] = np.array([val])
                
    #return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

    return pd.DataFrame([x["_source"] for x in elastic_docs])

## Lectura de datos ohmyfi-detalleconexiones

Los datos que se toman son:

fechahora (de cada conexión). fecha_control es lo mismo pero con el intervalo de 10 minutos
Información del centro: lugar, lugar_cod.
Información del dispositivo de usuario: mac_usuario, dispositivo, sistema_operativo.
Información de usuario: tipodoc y documento

In [18]:
def trae_conexiones(fecha_ini,fecha_fin):
    total_docs = 5000000
    response = es.search(
        index= parametros.ohmyfi_d_c_index,
        body={
                "_source": ["fechahora","fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo"
                            ,"sistema_operativo",'tipodoc','documento']
                , "query": {
                  "range": {
                    "fechahora": {
                      "gte": fecha_ini,
                      "lt": fecha_fin
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

    return pd.DataFrame([x["_source"] for x in elastic_docs])

In [19]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)

if datos_det_conex is None or datos_det_conex.empty:
    while (datos_det_conex is None or datos_det_conex.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [20]:
datos_det_conex['fecha'] = datos_det_conex['fecha_control'].str.split(" ", n = 1, expand = True)[0]
datos_det_conex.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

In [21]:
datos_det_conex = datos_det_conex.rename(columns={'lugar_cod' : 'site_id'
                                                             ,'fechahora':'usuarios.fechaConexionUsuarios'
                                                             ,'dispositivo': 'usuarios.tipoDispositivoUsuarios'
                                                             , 'sistema_operativo': 'usuarios.sistemaOperativoUsuarios'})

Se corrigen valores errados de site_id en detalle conexiones

In [22]:
datos_det_conex['site_id'] = datos_det_conex['site_id'].str.replace("_","-")

### Se lee la información de cambium device client

Esta lectura se usa para identificar:

Lecturas de consumo por dispositivo (radios rx y tx)
Fabricante del dispositivo
la mac del ap que luego se usa para identificar el ap group(Indoor/Outdoor)

In [23]:
t1=time.time()

total_docs = 30000000
response = es.search(
    index= parametros.cambium_d_c_index,
    body={
            "_source": ["mac","ap_mac", 'manufacturer','name',"radio.band",'radio.rx_bytes','radio.tx_bytes'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"mac"
                    }
                  }
                  ]
                }
              }
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
# fields = {}
# for num, doc in enumerate(elastic_docs):
#     source_data = doc["_source"]
#     for key, val in source_data.items():
#         try:
#             fields[key] = np.append(fields[key], val)
#         except KeyError:
#             fields[key] = np.array([val])

# datos_dev_clients = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

datos_dev_clients = pd.DataFrame([x["_source"] for x in elastic_docs])

t2=time.time()

se cruza por mac_usuario
Los merge usan left para evitar perdida de datos en cruce con cambium-devicedevices. Aquellos datos que no cruzan se les marca como no identificados. En condiciones ideales, no debería presentarse ausencia de información
Solo se toma lo que cruza con INNER
se cambia el fabricante [Local MAC]
se cambian lo nulos

In [44]:
try:
    datos_dev_clients.drop_duplicates(inplace=True)
    datos_dev_clients = datos_dev_clients.rename(columns={'ap_mac' : 'usuarios.macRed','mac' : 'mac_usuario'})
    datos_dev_clients['manufacturer'] = datos_dev_clients['manufacturer'].replace("[Local MAC]", "No identificado")
    datos_dev_clients.fillna({'manufacturer':'No identificado'},inplace=True)
    datos_det_conex = pd.merge(datos_det_conex,  datos_dev_clients, on='mac_usuario',how='left')
    datos_det_conex = pd.merge(datos_det_conex,datos_dev[['usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    datos_det_conex.fillna({'usuarios.tipoDispositivoUsuarios':'No identificado'
                       ,'usuarios.sistemaOperativoUsuarios':'No identificado'
                       ,'manufacturer':'No identificado'
                       ,'radio.band':'No identificado'
                       },inplace=True)
except:
    pass

# Se ejecuta consulta de datos

Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
Se ejecuta la función de consulta con el rango de fechas.
Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [45]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)

if datos_consumos is None or datos_consumos.empty:
    #while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
    while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

# Marcando si usuario es nuevo o recurrente¶

Se toma del dataframe datos_det_conex y se cuenta las veces que cada mac_usuario aparece
Luego se suma con el historico de conexiones por usuario (indice intermedio ohmyfi-total-conexiones-historico)
marca como Nuevo(1 conexión).
De este proceso se obtiene el campo:

usuarios.usuariosNuevos
usuarios.totales.usuariosNuevos

In [46]:
try:
    datos_recurrencia = datos_det_conex[["fecha","tipodoc","documento"]].groupby(["tipodoc","documento"]).agg(['count']).reset_index()
    datos_recurrencia.columns = datos_recurrencia.columns.droplevel(1)
    datos_recurrencia= datos_recurrencia.rename(columns={'fecha' : 'usuarios_recurrencia_aux'})
    datos_recurrencia['tipodoc'] = datos_recurrencia['tipodoc'].replace('','No especificado')
    datos_recurrencia['documento'] = datos_recurrencia['documento'].replace('','No especificado')
except:
    pass

### Se lee historico

Se lee ohmyfi-total-conexiones-historico, el cual es un indice creado a partir de los datos crudos. Este guarda la cantidad de conexiones realizadas por ese usuario. Para que la consulta funcione correctamente se debe colocar el filtro que usuarios_recurrencia exista.



In [47]:
try:
    total_docs = 30000000
    response = es.search(
        index= parametros.ohmyfi_total_c_index,
        body={
              "_source": ['tipodoc', 'documento', 'usuarios_recurrencia'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"usuarios_recurrencia"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#    fields = {}
#    for num, doc in enumerate(elastic_docs):
#       source_data = doc["_source"]
#       for key, val in source_data.items():
#            try:
#                fields[key] = np.append(fields[key], val)
#            except KeyError:
#               fields[key] = np.array([val])

    #ya_en_indice = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    ya_en_indice = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    ya_en_indice = pd.DataFrame(columns=['tipodoc', 'documento', 'usuarios_recurrencia'])

In [48]:
ya_en_indice

,usuarios_recurrencia,tipodoc,documento
0,90.0,CC,1042999676
1,26.0,CC,1063481153
2,8.0,CC,1073986488
3,6.0,CC,1101177429
4,52.0,CC,1839473
...,...,...,...
259484,220.0,x,72017832
259485,176.0,x,72017915
259486,129.0,x,72018216
259487,132.0,x,72020959


In [49]:
if ya_en_indice is None or ya_en_indice.empty:
    datos_recurrencia= datos_recurrencia.rename(columns={'usuarios_recurrencia_aux' : 'usuarios_recurrencia'})
else:
    datos_recurrencia = pd.merge(datos_recurrencia, ya_en_indice, on=['tipodoc', 'documento'],how='left')
    datos_recurrencia.fillna({'usuarios_recurrencia':0},inplace=True)
    datos_recurrencia['usuarios_recurrencia'] = datos_recurrencia['usuarios_recurrencia'] + datos_recurrencia['usuarios_recurrencia_aux']
    datos_recurrencia['usuarios_recurrencia'] = [int(x) for x in datos_recurrencia['usuarios_recurrencia']]

In [50]:
try:
    aux_recurrencia = pd.merge(datos_recurrencia,datos_det_conex, on=['tipodoc','documento'],how='inner' )
    aux_recurrencia = aux_recurrencia[(aux_recurrencia['usuarios_recurrencia']==1)]
    aux_recurrencia.loc[aux_recurrencia['usuarios_recurrencia']==1,'usuarios.usuariosNuevos']='Nuevo'
    #aux_recurrencia.loc[aux_recurrencia['usuarios_recurrencia']!=1,'usuarios.usuariosNuevos']='Recurrente'
    aux_recurrencia = aux_recurrencia.rename(columns={'usuarios_recurrencia' :'usuarios.sesiones_Usuarios'})
    aux_recurrencia = aux_recurrencia[["fecha_control","site_id","usuarios.usuariosNuevos",'documento']].groupby(["fecha_control","site_id","usuarios.usuariosNuevos"])['documento'].nunique().reset_index()
    aux_recurrencia= aux_recurrencia.rename(columns={'documento' : 'usuarios.totales.usuariosNuevos'})
except:
    pass

In [51]:
try:
    datos_recurrencia = pd.merge(datos_semilla,  aux_recurrencia, on='site_id', how='inner')
    datos_recurrencia = datos_recurrencia.rename(columns={'fecha_control' : 'usuarios.fechaControl'})
    datos_recurrencia["usuarios.fecha"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    datos_recurrencia["usuarios.anyo"] = datos_recurrencia["usuarios.fecha"].str[0:4]
    datos_recurrencia["usuarios.mes"] = datos_recurrencia["usuarios.fecha"].str[5:7]
    datos_recurrencia["usuarios.dia"] = datos_recurrencia["usuarios.fecha"].str[8:10]
    datos_recurrencia["usuarios.hora"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    datos_recurrencia["usuarios.minuto"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    datos_recurrencia= datos_recurrencia.rename(columns={'site_id' : 'usuarios.siteID'})
    
    datos_recurrencia['nombreDepartamento'] = datos_recurrencia['usuarios.nombreDepartamento']
    datos_recurrencia['nombreMunicipio'] = datos_recurrencia['usuarios.nombreMunicipio']
    datos_recurrencia['idBeneficiario'] = datos_recurrencia['usuarios.idBeneficiario']
    datos_recurrencia['fecha'] = datos_recurrencia['usuarios.fecha']
    datos_recurrencia['anyo'] = datos_recurrencia['usuarios.anyo']
    datos_recurrencia['mes'] = datos_recurrencia['usuarios.mes']
    datos_recurrencia['dia'] = datos_recurrencia['usuarios.dia']
except:
    pass

In [52]:
datos_recurrencia

,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.siteID,usuarios.nombreDepartamento,usuarios.sistemaEnergia,usuarios.nombreMunicipio,usuarios.idBeneficiario,usuarios.location,usuarios.fechaControl,...,usuarios.dia,usuarios.hora,usuarios.minuto,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia
0,ESC NVA PIEDRA ABAJO,CO-COR,VDA PIEDRAS ABAJO,31505-ZZZY232,CÓRDOBA,RED INTERCONECTADA,CIÉNAGA DE ORO,31505,"7.9471111,-73.656805000",2021-06-07 00:00:00,...,07,00,00,CÓRDOBA,CIÉNAGA DE ORO,31505,2021-06-07,2021,06,07
1,SEDE PRINCIPAL,CO-COR,PUEBLECITO SUR,32284-ZZZY240,CÓRDOBA,RED INTERCONECTADA,SAN ANDRÉS DE SOTAVENTO,32284,"9.3517975,-75.79683054",2021-06-07 00:00:00,...,07,00,00,CÓRDOBA,SAN ANDRÉS DE SOTAVENTO,32284,2021-06-07,2021,06,07
2,I.E. DON ALONSO - SEDE PRINCIPAL,CO-SUC,DON ALONSO,47119-ZZZY211,SUCRE,RED INTERCONECTADA,COROZAL,47119,"9.25824627,-75.31213588",2021-06-07 00:00:00,...,07,00,00,SUCRE,COROZAL,47119,2021-06-07,2021,06,07
3,JAGUALITO PUEBLO NUEVO,CO-TOL,PUEBLO NUEVO,48605-ZGYO579,TOLIMA,RED INTERCONECTADA,GUAMO,48605,"4.000838889,-74.894555556",2021-06-07 00:00:00,...,07,00,00,TOLIMA,GUAMO,48605,2021-06-07,2021,06,07


In [53]:
datos_recurrencia.columns

Index(['usuarios.nomCentroDigital', 'usuarios.codISO', 'usuarios.localidad',
       'usuarios.siteID', 'usuarios.nombreDepartamento',
       'usuarios.sistemaEnergia', 'usuarios.nombreMunicipio',
       'usuarios.idBeneficiario', 'usuarios.location', 'usuarios.fechaControl',
       'usuarios.usuariosNuevos', 'usuarios.totales.usuariosNuevos',
       'usuarios.fecha', 'usuarios.anyo', 'usuarios.mes', 'usuarios.dia',
       'usuarios.hora', 'usuarios.minuto', 'nombreDepartamento',
       'nombreMunicipio', 'idBeneficiario', 'fecha', 'anyo', 'mes', 'dia'],
      dtype='object')

# Total registros  a indice

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [103]:
use_these_keys = ['usuarios.fecha'
                  ,'usuarios.siteID'
                  ,'usuarios.usuariosRegistrados'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.location'
                  , 'usuarios.macRed'
                  , 'usuarios.fechaControl'
                  , 'usuarios.tiempoPromedioSesionSitio'
                  , 'usuarios.apGroup'
                  , 'usuarios.totales.usuariosNuevos'
                  , 'usuarios.sesiones_Usuarios'
                  , 'usuarios.usuariosConectados'
                  #, 'usuarios.hora'
                  #, 'usuarios.minuto'
                  , 'ip'
                  , 'status'
                  , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        document = document.rename(index={ 'usuarios.apGroup_x':'usuarios.apGroup', 'usuarios.fecha_x': 'usuarios.fecha','usuarios.anyo_x': 'usuarios.anyo','usuarios.mes_x':'usuarios.mes','usuarios.dia_x':'usuarios.dia','usuarios.nomCentroDigital_x':'usuarios.nomCentroDigital', 'usuarios.codISO_x':'usuarios.codISO', 'usuarios.localidad_x':'usuarios.localidad', 'usuarios.nombreDepartamento_x':'usuarios.nombreDepartamento', 'usuarios.sistemaEnergia_x':'usuarios.sistemaEnergia', 'usuarios.nombreMunicipio_x':'usuarios.nombreMunicipio', 'usuarios.idBeneficiario_x':'usuarios.idBeneficiario', 'usuarios.location_x':'usuarios.location', 'nombreDepartamento_x':'nombreDepartamento', 'nombreMunicipio_x':'nombreMunicipio', 'idBeneficiario_x':'idBeneficiario', 'usuarios.fechaControl_x':'usuarios.fechaControl', 'usuarios.macRed_x':'usuarios.macRed','fecha_x':'fecha', 'anyo_x':'anyo', 'mes_x':'mes', 'dia_x':'dia', '@timestamp_x':'@timestamp'})
        
        #print(document)
        #print("=========================")
        yield {
                "_index": indice, 
                "_id": f"{'UsuRegistro-'+ str(document['usuarios.siteID']) + '-' + str(document['usuarios.fecha']) + str(random.randrange(1000))}",
                "_source": filterKeys(document),
            }

Para cada sitio y fecha, se cuentan los documentos de usuario. Esto resulta en el total de usuarios registrados para esa fecha

In [104]:
def invalid_values(df, values=['']):
    return df.loc[df['usuarios.location'].isin(values)]

In [105]:
try:
    
    datos_registro['creado'] = datos_registro['creado'].str[0:10]
    datos_registro['tipo_mas_doc'] = datos_registro['tipodoc'] + datos_registro['documento']
    datos_registro= datos_registro.rename(columns={'creado' : 'usuarios.fecha'
                                                  ,'lugar_cod':'site_id'})

    datos_registro = datos_registro[["usuarios.fecha","site_id","tipo_mas_doc"]].groupby(["usuarios.fecha","site_id"]).agg(['count']).reset_index()
    datos_registro.columns = datos_registro.columns.droplevel(1)
    datos_registro= datos_registro.rename(columns={'tipo_mas_doc' : 'usuarios.usuariosRegistrados'})
    datos_registro["usuarios.anyo"] = datos_registro["usuarios.fecha"].str[0:4]
    datos_registro["usuarios.mes"] = datos_registro["usuarios.fecha"].str[5:7]
    datos_registro["usuarios.dia"] = datos_registro["usuarios.fecha"].str[8:10]
    datos_registro = pd.merge(datos_registro,  datos_semilla, on='site_id', how='inner')
    datos_registro.rename(columns={'site_id': 'usuarios.siteID'
                               }, inplace=True)
    
    datos_registro['nombreDepartamento'] = datos_registro['usuarios.nombreDepartamento']
    datos_registro['nombreMunicipio'] = datos_registro['usuarios.nombreMunicipio']
    datos_registro['idBeneficiario'] = datos_registro['usuarios.idBeneficiario']
    datos_registro['fecha'] = datos_registro['usuarios.fecha']
    datos_registro['anyo'] = datos_registro['usuarios.anyo']
    datos_registro['mes'] = datos_registro['usuarios.mes']
    datos_registro['dia'] = datos_registro['usuarios.dia']
    
    datos_registro['@timestamp'] = now.isoformat()
    #salida = helpers.bulk(es, doc_generator(datos_registro))
    #print("Fecha: ", now,"- Datos Registro usuarios en indice principal:",salida[0])
except Exception as e:
    print("Fecha: ", now,"- Nada que insertar en indice principal",e)

Fecha:  2021-08-09 22:26:01.102495 - Nada que insertar en indice principal 'creado'


In [106]:
datos_registro.head()

,usuarios.fecha,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo,usuarios.mes,usuarios.dia,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.nombreDepartamento,...,usuarios.idBeneficiario,usuarios.location,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia,@timestamp
0,2021-06-07,10835-ZGYO372,2,2021,06,07,E.U. SIETE DE AGOSTO,CO-ANT,CALLE 115 # 06 - 60,ANTIOQUIA,...,10835,"10.88307108,-75.077732850",ANTIOQUIA,TURBO,10835,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
1,2021-06-07,11107-ZGYO062,1,2021,06,07,MARIA INMACULADA,CO-ATL,CALLE 15 12 - 20,ATLÁNTICO,...,11107,"10.504888,-74.816395",ATLÁNTICO,BARANOA,11107,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
2,2021-06-07,11128-ZGYO932,2,2021,06,07,LAS PETRONITAS,CO-ATL,VEREDA LAS PETRONITAS CORDIALIDAD,ATLÁNTICO,...,11128,"8.751218,-75.49306166",ATLÁNTICO,GALAPA,11128,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
3,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,13253,"3.985,-75.06861111",CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
4,2021-06-07,13258-ZGYO980,15,2021,06,07,IE ANTONIO ENRIQUE DIAZ MARTINEZ,CO-CES,BADILLO,CESAR,...,13258,"10.40971209,-73.5167746",CESAR,VALLEDUPAR,13258,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495


In [107]:
datos_registro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 0 to 358
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   usuarios.fecha                359 non-null    object
 1   usuarios.siteID               359 non-null    object
 2   usuarios.usuariosRegistrados  359 non-null    int64 
 3   usuarios.anyo                 359 non-null    object
 4   usuarios.mes                  359 non-null    object
 5   usuarios.dia                  359 non-null    object
 6   usuarios.nomCentroDigital     359 non-null    object
 7   usuarios.codISO               359 non-null    object
 8   usuarios.localidad            359 non-null    object
 9   usuarios.nombreDepartamento   359 non-null    object
 10  usuarios.sistemaEnergia       359 non-null    object
 11  usuarios.nombreMunicipio      359 non-null    object
 12  usuarios.idBeneficiario       359 non-null    int64 
 13  usuarios.location   

In [108]:
try:
    datos_consumos = datos_consumos.rename(columns={'lugar_cod' :'site_id'})
    datos_consumos['mac_usuario'] = datos_consumos['mac_usuario'].str.replace('-',':')
    datos_consumos['fecha_control'] = datos_consumos["fecha_inicio"].str[0:-4] + '0:00'
    tiempoPromedioSesionSitio=datos_consumos[['site_id','mac_usuario','fecha_control','tiempo_sesion_minutos']].groupby(['site_id','mac_usuario','fecha_control']).agg(['mean']).reset_index()
    tiempoPromedioSesionSitio.columns = tiempoPromedioSesionSitio.columns.droplevel(1)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'tiempo_sesion_minutos' : 'usuarios.tiempoPromedioSesionSitio'})
    tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'] = round(tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'],6)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'mac_usuario' : 'usuarios.macRed'
                                                                     , 'fecha_control' : 'usuarios.fechaControl'})
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio,datos_semilla, on ='site_id', how='inner')
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio, datos_dev, on=['site_id','usuarios.macRed'], how='left')
    tiempoPromedioSesionSitio.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'site_id' : 'usuarios.siteID'})
    tiempoPromedioSesionSitio["usuarios.fecha"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str[0:10]
    tiempoPromedioSesionSitio["usuarios.anyo"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[0:4]
    tiempoPromedioSesionSitio["usuarios.mes"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[5:7]
    tiempoPromedioSesionSitio["usuarios.dia"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[8:10]
    tiempoPromedioSesionSitio["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    tiempoPromedioSesionSitio["usuarios.minuto"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    tiempoPromedioSesionSitio['nombreDepartamento'] = tiempoPromedioSesionSitio['usuarios.nombreDepartamento']
    tiempoPromedioSesionSitio['nombreMunicipio'] = tiempoPromedioSesionSitio['usuarios.nombreMunicipio']
    tiempoPromedioSesionSitio['idBeneficiario'] = tiempoPromedioSesionSitio['usuarios.idBeneficiario']
    tiempoPromedioSesionSitio['fecha'] = tiempoPromedioSesionSitio['usuarios.fecha']
    tiempoPromedioSesionSitio['anyo'] = tiempoPromedioSesionSitio['usuarios.anyo']
    tiempoPromedioSesionSitio['mes'] = tiempoPromedioSesionSitio['usuarios.mes']
    tiempoPromedioSesionSitio['dia'] = tiempoPromedioSesionSitio['usuarios.dia']
    
    tiempoPromedioSesionSitio['@timestamp'] = now.isoformat()
    #salida = helpers.bulk(es, doc_generator(tiempoPromedioSesionSitio))
    #print("Fecha: ", now,"- Tiempo promedio sesion en sitio insertado en indice principal:",salida[0])
except Exception as e:
    print("Fecha: ", now,"- Nada para insertar en indice principal",e)

In [109]:
tiempoPromedioSesionSitio.head()

,usuarios.siteID,usuarios.macRed,usuarios.fechaControl,usuarios.tiempoPromedioSesionSitio,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.nombreDepartamento,usuarios.sistemaEnergia,usuarios.nombreMunicipio,...,usuarios.hora,usuarios.minuto,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia,@timestamp
0,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:10:00,5.47,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,10,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
1,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:20:00,4.53,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,20,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
2,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:30:00,12.02,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
3,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:40:00,3.27,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,40,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
4,13253-ZGYO527,D6:7A:05:FA:CD:D3,2021-06-07 00:30:00,1.87,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495


In [110]:
tiempoPromedioSesionSitio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326 entries, 0 to 325
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   usuarios.siteID                     326 non-null    object 
 1   usuarios.macRed                     326 non-null    object 
 2   usuarios.fechaControl               326 non-null    object 
 3   usuarios.tiempoPromedioSesionSitio  326 non-null    float64
 4   usuarios.nomCentroDigital           326 non-null    object 
 5   usuarios.codISO                     326 non-null    object 
 6   usuarios.localidad                  326 non-null    object 
 7   usuarios.nombreDepartamento         326 non-null    object 
 8   usuarios.sistemaEnergia             326 non-null    object 
 9   usuarios.nombreMunicipio            326 non-null    object 
 10  usuarios.idBeneficiario             326 non-null    int64  
 11  usuarios.location                   326 non-n

In [111]:
# Insertando usuarios conectados al indice principal

In [112]:
try:
    datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento','usuarios.macRed']].groupby(["fecha_control","site_id","usuarios.macRed"])['documento'].nunique().reset_index()
    datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})
    usuariosConectados = datos_det_conex[["fecha_control","site_id","usuarios.macRed","mac_usuario"]].groupby(["fecha_control","site_id","usuarios.macRed"]).agg(['count']).reset_index()
    usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
    usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.usuariosConectados'})
    usuariosConectados = pd.merge(usuariosConectados,datos_logins, on=["fecha_control","site_id","usuarios.macRed"], how='inner')
    usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
    usuariosConectados.fillna({'usuarios.consumoUsuarios' : 0
                              ,'usuarios.consumoUsuariosDescarga':0
                              ,'usuarios.consumoUsuariosCarga':0},inplace=True)
    usuariosConectados = pd.merge(usuariosConectados,datos_dev[['usuarios.apGroup','usuarios.macRed']], on ='usuarios.macRed', how='left')
    usuariosConectados.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    usuariosConectados['usuarios.usuariosConectados'] = usuariosConectados['usuarios.usuariosConectados'].astype(int)
    usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    usuariosConectados["usuarios.fecha"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    usuariosConectados["usuarios.anyo"] = usuariosConectados["usuarios.fecha"].str[0:4]
    usuariosConectados["usuarios.mes"] = usuariosConectados["usuarios.fecha"].str[5:7]
    usuariosConectados["usuarios.dia"] = usuariosConectados["usuarios.fecha"].str[8:10]
    usuariosConectados["usuarios.hora"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    usuariosConectados["usuarios.minuto"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    usuariosConectados['nombreDepartamento'] = usuariosConectados['usuarios.nombreDepartamento']
    usuariosConectados['nombreMunicipio'] = usuariosConectados['usuarios.nombreMunicipio']
    usuariosConectados['idBeneficiario'] = usuariosConectados['usuarios.idBeneficiario']
    usuariosConectados['fecha'] = usuariosConectados['usuarios.fecha']
    usuariosConectados['anyo'] = usuariosConectados['usuarios.anyo']
    usuariosConectados['mes'] = usuariosConectados['usuarios.mes']
    usuariosConectados['dia'] = usuariosConectados['usuarios.dia']
except:
    pass

In [113]:

datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento','usuarios.macRed']].groupby(["fecha_control","site_id","usuarios.macRed"])['documento'].nunique().reset_index()
datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})
usuariosConectados = datos_det_conex[["fecha_control","site_id","usuarios.macRed","mac_usuario"]].groupby(["fecha_control","site_id","usuarios.macRed"]).agg(['count']).reset_index()
usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.usuariosConectados'})
usuariosConectados = pd.merge(usuariosConectados,datos_logins, on=["fecha_control","site_id","usuarios.macRed"], how='inner')
usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
usuariosConectados.fillna({'usuarios.consumoUsuarios' : 0
                          ,'usuarios.consumoUsuariosDescarga':0
                          ,'usuarios.consumoUsuariosCarga':0},inplace=True)
usuariosConectados = pd.merge(usuariosConectados,datos_dev[['usuarios.apGroup','usuarios.macRed']], on ='usuarios.macRed', how='left')
usuariosConectados.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
usuariosConectados['usuarios.usuariosConectados'] = usuariosConectados['usuarios.usuariosConectados'].astype(int)
usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                       ,'site_id' : 'usuarios.siteID'})

usuariosConectados["usuarios.fecha"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
usuariosConectados["usuarios.anyo"] = usuariosConectados["usuarios.fecha"].str[0:4]
usuariosConectados["usuarios.mes"] = usuariosConectados["usuarios.fecha"].str[5:7]
usuariosConectados["usuarios.dia"] = usuariosConectados["usuarios.fecha"].str[8:10]
usuariosConectados["usuarios.hora"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
usuariosConectados["usuarios.minuto"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]

usuariosConectados['nombreDepartamento'] = usuariosConectados['usuarios.nombreDepartamento']
usuariosConectados['nombreMunicipio'] = usuariosConectados['usuarios.nombreMunicipio']
usuariosConectados['idBeneficiario'] = usuariosConectados['usuarios.idBeneficiario']
usuariosConectados['fecha'] = usuariosConectados['usuarios.fecha']
usuariosConectados['anyo'] = usuariosConectados['usuarios.anyo']
usuariosConectados['mes'] = usuariosConectados['usuarios.mes']
usuariosConectados['dia'] = usuariosConectados['usuarios.dia']


In [114]:
datos_logins.columns

Index(['fecha_control', 'site_id', 'usuarios.macRed',
       'usuarios.sesiones_Usuarios'],
      dtype='object')

In [136]:
 result1=pd.merge(datos_registro, tiempoPromedioSesionSitio,on=['usuarios.siteID'],how='inner')

In [137]:
 result1

,usuarios.fecha_x,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo_x,usuarios.mes_x,usuarios.dia_x,usuarios.nomCentroDigital_x,usuarios.codISO_x,usuarios.localidad_x,usuarios.nombreDepartamento_x,...,usuarios.hora,usuarios.minuto,nombreDepartamento_y,nombreMunicipio_y,idBeneficiario_y,fecha_y,anyo_y,mes_y,dia_y,@timestamp_y
0,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,10,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
1,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,20,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
2,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
3,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,40,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
4,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,2021-06-07,73868-ZGYO254,1,2021,06,07,CENTRO ETNOEDUCATIVO RURAL SAN LORENZO DE SHIR...,CO-LAG,SHIRURI,LA GUAJIRA,...,00,40,LA GUAJIRA,MANAURE,73868,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
288,2021-06-07,73868-ZGYO254,1,2021,06,07,CENTRO ETNOEDUCATIVO RURAL SAN LORENZO DE SHIR...,CO-LAG,SHIRURI,LA GUAJIRA,...,00,10,LA GUAJIRA,MANAURE,73868,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
289,2021-06-07,73868-ZGYO254,1,2021,06,07,CENTRO ETNOEDUCATIVO RURAL SAN LORENZO DE SHIR...,CO-LAG,SHIRURI,LA GUAJIRA,...,00,00,LA GUAJIRA,MANAURE,73868,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495
290,2021-06-07,73868-ZGYO254,1,2021,06,07,CENTRO ETNOEDUCATIVO RURAL SAN LORENZO DE SHIR...,CO-LAG,SHIRURI,LA GUAJIRA,...,00,10,LA GUAJIRA,MANAURE,73868,2021-06-07,2021,06,07,2021-08-09T22:26:01.102495


In [126]:
 result2=pd.merge(result1, usuariosConectados,on=['usuarios.siteID'],how='inner')

In [127]:
result2

,usuarios.fecha_x,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo_x,usuarios.mes_x,usuarios.dia_x,usuarios.nomCentroDigital_x,usuarios.codISO_x,usuarios.localidad_x,usuarios.nombreDepartamento_x,...,usuarios.dia,usuarios.hora_y,usuarios.minuto_y,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia
0,2021-06-07,20006-ZGYO924,3,2021,06,07,I. E. LLANO GRANDE,CO-ANT,CORREG. MARSELLA,ANTIOQUIA,...,07,00,00,ANTIOQUIA,FREDONIA,20006,2021-06-07,2021,06,07
1,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
2,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
3,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
4,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
5,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
6,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
7,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
8,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07
9,2021-06-07,26489-ZGYO602,8,2021,06,07,INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...,CO-CAL,SAN JERONIMO,CALDAS,...,07,00,00,CALDAS,RIOSUCIO,26489,2021-06-07,2021,06,07


In [144]:
result=pd.merge(result2, datos_recurrencia,on=['usuarios.siteID'],how='inner')

In [145]:
result

,usuarios.fecha_x,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo_x,usuarios.mes_x,usuarios.dia_x,usuarios.nomCentroDigital_x,usuarios.codISO_x,usuarios.localidad_x,usuarios.nombreDepartamento_x,...,usuarios.dia_y,usuarios.hora,usuarios.minuto,nombreDepartamento_y,nombreMunicipio_y,idBeneficiario_y,fecha_y,anyo_y,mes_y,dia_y


In [146]:
datos_recurrencia

,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.siteID,usuarios.nombreDepartamento,usuarios.sistemaEnergia,usuarios.nombreMunicipio,usuarios.idBeneficiario,usuarios.location,usuarios.fechaControl,...,usuarios.dia,usuarios.hora,usuarios.minuto,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia
0,ESC NVA PIEDRA ABAJO,CO-COR,VDA PIEDRAS ABAJO,31505-ZZZY232,CÓRDOBA,RED INTERCONECTADA,CIÉNAGA DE ORO,31505,"7.9471111,-73.656805000",2021-06-07 00:00:00,...,07,00,00,CÓRDOBA,CIÉNAGA DE ORO,31505,2021-06-07,2021,06,07
1,SEDE PRINCIPAL,CO-COR,PUEBLECITO SUR,32284-ZZZY240,CÓRDOBA,RED INTERCONECTADA,SAN ANDRÉS DE SOTAVENTO,32284,"9.3517975,-75.79683054",2021-06-07 00:00:00,...,07,00,00,CÓRDOBA,SAN ANDRÉS DE SOTAVENTO,32284,2021-06-07,2021,06,07
2,I.E. DON ALONSO - SEDE PRINCIPAL,CO-SUC,DON ALONSO,47119-ZZZY211,SUCRE,RED INTERCONECTADA,COROZAL,47119,"9.25824627,-75.31213588",2021-06-07 00:00:00,...,07,00,00,SUCRE,COROZAL,47119,2021-06-07,2021,06,07
3,JAGUALITO PUEBLO NUEVO,CO-TOL,PUEBLO NUEVO,48605-ZGYO579,TOLIMA,RED INTERCONECTADA,GUAMO,48605,"4.000838889,-74.894555556",2021-06-07 00:00:00,...,07,00,00,TOLIMA,GUAMO,48605,2021-06-07,2021,06,07


In [147]:
try:
    result1=pd.merge(datos_registro, tiempoPromedioSesionSitio,on=['usuarios.siteID'],how='inner')
    result2=pd.merge(result1, usuariosConectados,on=['usuarios.siteID'],how='inner')
    result=pd.merge(result2, datos_recurrencia,on=['usuarios.siteID'],how='left')
    #salida = helpers.bulk(es, doc_generator(result,use_these_keys))
    result.fillna('', inplace=True)
    #result['usuarios.fechaControl'] = fecha_ejecucion
    result['usuarios.fecha_x'] = fecha_ejecucion[0:4] + '-' + fecha_ejecucion[5:7] + '-' + fecha_ejecucion[8:10]
    #result['usuarios.location_x'] = "0.0000000,0.0000000"
    
    salida = helpers.bulk(es, doc_generator(result))
    print("Fecha: ", now,"- Cantidad de registros insertado en indice principal:",salida[0])
except Exception as e:
    print(e)
    print("Fecha: ", now,"- Nada para insertar en indice principal")

('22 document(s) failed to index.', [{'index': {'_index': 'dev-usuarios-tablero12-001', '_type': '_doc', '_id': 'UsuRegistro-20006-ZGYO924-usuarios.fecha    2021-06-07\nusuarios.fecha    2021-06-07\nName: 0, dtype: object389', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': "failed to parse field [usuarios.fechaControl] of type [date] in document with id 'UsuRegistro-20006-ZGYO924-usuarios.fecha    2021-06-07\nusuarios.fecha    2021-06-07\nName: 0, dtype: object389'. Preview of field's value: ''", 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'cannot parse empty date'}}, 'data': {'usuarios.fecha': usuarios.fecha    2021-06-07
usuarios.fecha    2021-06-07
Name: 0, dtype: object, 'usuarios.siteID': '20006-ZGYO924', 'usuarios.usuariosRegistrados': 3, 'usuarios.anyo': usuarios.anyo    2021
usuarios.anyo    2021
Name: 0, dtype: object, 'usuarios.mes': usuarios.mes    06
usuarios.mes    06
Name: 0, dtype: object, 'usuarios.dia': usuarios.dia    07
usuar

In [148]:
result.columns

Index(['usuarios.fecha_x', 'usuarios.siteID', 'usuarios.usuariosRegistrados',
       'usuarios.anyo_x', 'usuarios.mes_x', 'usuarios.dia_x',
       'usuarios.nomCentroDigital_x', 'usuarios.codISO_x',
       'usuarios.localidad_x', 'usuarios.nombreDepartamento_x',
       'usuarios.sistemaEnergia_x', 'usuarios.nombreMunicipio_x',
       'usuarios.idBeneficiario_x', 'usuarios.location_x',
       'nombreDepartamento_x', 'nombreMunicipio_x', 'idBeneficiario_x',
       'fecha_x', 'anyo_x', 'mes_x', 'dia_x', '@timestamp_x',
       'usuarios.macRed_x', 'usuarios.fechaControl_x',
       'usuarios.tiempoPromedioSesionSitio', 'usuarios.nomCentroDigital_y',
       'usuarios.codISO_y', 'usuarios.localidad_y',
       'usuarios.nombreDepartamento_y', 'usuarios.sistemaEnergia_y',
       'usuarios.nombreMunicipio_y', 'usuarios.idBeneficiario_y',
       'usuarios.location_y', 'usuarios.apGroup_x', 'ip', 'status',
       'usuarios.fecha_y', 'usuarios.anyo_y', 'usuarios.mes_y',
       'usuarios.dia_y', 'us

In [88]:
#salida = helpers.bulk(es, doc_generator(result))

In [121]:
result

,usuarios.fecha_x,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo_x,usuarios.mes_x,usuarios.dia_x,usuarios.nomCentroDigital_x,usuarios.codISO_x,usuarios.localidad_x,usuarios.nombreDepartamento_x,...,usuarios.dia_y,usuarios.hora,usuarios.minuto,nombreDepartamento_y,nombreMunicipio_y,idBeneficiario_y,fecha_y,anyo_y,mes_y,dia_y


In [120]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 100 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   usuarios.fecha_x                    0 non-null      object 
 1   usuarios.siteID                     0 non-null      object 
 2   usuarios.usuariosRegistrados        0 non-null      int64  
 3   usuarios.anyo_x                     0 non-null      object 
 4   usuarios.mes_x                      0 non-null      object 
 5   usuarios.dia_x                      0 non-null      object 
 6   usuarios.nomCentroDigital_x         0 non-null      object 
 7   usuarios.codISO_x                   0 non-null      object 
 8   usuarios.localidad_x                0 non-null      object 
 9   usuarios.nombreDepartamento_x       0 non-null      object 
 10  usuarios.sistemaEnergia_x           0 non-null      object 
 11  usuarios.nombreMunicipio_x          0 non-null      object 
 

## Actualiza fecha control

* Se actualiza la fecha de control. Si el calculo supera la fecha actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion= (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero12',
        body = { 'jerarquia-tablero12': 'jerarquia-tablero12','usuarios.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)